# sequence_encoder

level追加

In [35]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass

import pandas as pd
import polars as pl
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


In [36]:
@dataclass
class Cfg:
    input_dir = "/mnt/predict-student-performance-from-game-play/input/"
    output_dir = "/mnt/predict-student-performance-from-game-play/output/"
    prep_dir = "/mnt/predict-student-performance-from-game-play/prep/"
cfg = Cfg()

In [37]:
level_groups = ['0-4', '5-12', '13-22']

In [38]:
train = pd.read_csv(cfg.input_dir + "train.csv")

In [39]:
room_fqids = train["room_fqid"].unique().tolist()
room_fqid_encoder = {room_fqid: str(i) for i,room_fqid in enumerate(room_fqids)}
room_fqid_decoder = {str(i): room_fqid for i,room_fqid in enumerate(room_fqids)}

In [40]:
sequence_encoders = {}
for group in level_groups:
    df = pl.from_pandas(train[train["level_group"]==group])
    df = df.with_columns(pl.col("room_fqid").map_dict(room_fqid_encoder).alias("room_fqid_encode"))
    df = df.groupby("session_id").agg(pl.col("room_fqid_encode").filter(pl.col("room_fqid_encode")!=pl.col("room_fqid_encode").shift(1)).str.concat(""))
    room_fqid_sequences = list(df["room_fqid_encode"].unique())
    room_fqid_sequence_encoder = {room_fqid_seq: i for i,room_fqid_seq in enumerate(room_fqid_sequences)}
    sequence_encoders[group] = {}
    sequence_encoders[group]["room_fqid"] = room_fqid_sequence_encoder

In [43]:
with open(cfg.prep_dir + "room_fqid_encoder.pkl", "wb") as f:
    pickle.dump(room_fqid_encoder, f)

with open(cfg.prep_dir + "sequence_encoders.pkl", "wb") as f:
    pickle.dump(sequence_encoders, f)

In [42]:
#df.groupby("session_id").agg(pl.col("room_fqid_encode").filter(pl.col("room_fqid_encode")!=pl.col("room_fqid_encode").shift(1)).str.concat("").map_dict(room_fqid_sequence_encoder))